In [5]:
# coding:utf-8
# author: Fengzhijin
# time: 2017.11.16
# ==================================
'''
运用神经网络解决FashionMNIST数据识别问题
1.get_weights_bases() - 生成各层权值及偏置值函数
2.hidden_inference() - 隐藏层计算函数
3.inference() - 输构建两层神经网络结构函数
4.train() - 神经网络训练
5.此算法对模型正确率加入了可视化
'''


import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 模型相关的参数
LEARNING_RATE_BASE = 0.1    # 初始化的学习率
LEARNING_RATE_DECAY = 0.99    # 学习率的衰减率
BATCH_SIZE = 100     # 每次batch打包的样本个数
TRAINING_STEPS = 30000    # 训练轮数

# 配置神经网络的参数
INPUT_NODE = 784     # 输入层的节点数
OUTPUT_NODE = 10     # 输出层的节点数
LAYER1_NODE = 784    # 隐藏层一节点数，有784个节点
LAYER2_NODE = 100    # 隐藏层二节点数，有100个节点
REGULARAZTION_RATE = 0.0001    # 描述模型复杂度的正则化项在损失函数中的系数


# 生成各层权值和偏置值
def get_weights_bases(shape):
    weights = tf.Variable(tf.truncated_normal([shape[0], shape[1]], stddev=0.1))
    bases = tf.Variable(tf.constant(0.1, shape=[shape[1]]))
    return weights, bases


# 隐藏层运算
def hidden_inference(input_tensor, weights, biases):
    layer = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
    return layer


# 输出层运算
def output_inference(input_tensor, weights, biases):
    layer = tf.matmul(input_tensor, weights) + biases
    return layer


# 构建两层神经网络模型
def inference(x, y_, variable_averages=None):
    # 生成隐藏层1的参数。
    weights1, biases1 = get_weights_bases([INPUT_NODE, LAYER1_NODE])
    # 生成隐藏层2的参数。
    weights2, biases2 = get_weights_bases([LAYER1_NODE, LAYER2_NODE])
    # 生成输出层的参数。
    weights3, biases3 = get_weights_bases([LAYER2_NODE, OUTPUT_NODE])
    # 隐藏层1的输出
    y1 = hidden_inference(x, weights1, biases1)
    # 隐藏层2的输出
    y2 = hidden_inference(y1, weights2, biases2)
    # 输出层结果
    y = output_inference(y2, weights3, biases3)
    # 使用TensorFlow中提供的sparse_softmax_cross_entropy_with_logits函数来计算交叉熵
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    # 计算当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
    # 计算模型的正则化损失。一般只计算神经网络边上权重的正则化损失，而不是用偏置项
    regularaztion = regularizer(weights1) + regularizer(weights2) + regularizer(weights3)
    # 总损失等于交叉熵损失和正则化损失的总和
    loss = cross_entropy_mean + regularaztion
    return y, loss


def train(mnist):
    with tf.Graph().as_default() as graph:
        # 生成输入数据集和标签集
        x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
        # 定义存储训练轮数的变量
        global_step = tf.Variable(0)
        # 计算前向传播结果和损失函数
        y, loss = inference(x, y_)

        # 设置指数衰减的学习率。
        learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,    # 初始化的学习率，随着迭代的进行，更新变量时使用的学习率在这个基础上递减
            global_step,    # 当前迭代的轮数
            mnist.train.num_examples / BATCH_SIZE,    # 过完所有的训练数据需要的迭代次数
            LEARNING_RATE_DECAY)    # 学习率衰减速度

        # 使用梯度下降优化算法来优化损失函数
        # train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(loss, global_step=global_step)
        train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(
            loss, global_step=global_step)
        # train_step = tf.train.AdamOptimizer().minimize(loss, global_step=global_step)

        # 更新神经网络中的参数
        with tf.control_dependencies([train_step]):
            train_op = tf.no_op(name='train')

        # 检验神经网络前向传播结果是否正确
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        # 计算这一组数据上的正确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        # 将accuracy正确率在tensorboard的SCALAR、HISTOGRAM面板上创建可视化摘要
        summary_op = tf.summary.scalar('value', accuracy)
        histogram = tf.summary.histogram('histogram', accuracy)

    # 初始化回话并开始训练过程。
    with tf.Session(graph=graph) as sess:
        # 创建事件文件
        writer = tf.summary.FileWriter('./graphs')
        writer.add_graph(graph)
        # 初始化所有变量
        tf.global_variables_initializer().run()
        # 准备验证数据
        validate_feed = {x: mnist.validation.images, y_:
                         mnist.validation.labels}
        # 准备测试数据
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}

        # 迭代地训练神经网络。
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证数据集上的测试结果
            if i % 1000 == 0:
                # 计算模型验证数据上的正确率
                # validate_acc = sess.run(accuracy, feed_dict=validate_feed)

                # 计算模型在验证数据集上的正确率并运行摘要
                summary, acc = sess.run([summary_op, accuracy], feed_dict=validate_feed)
                writer.add_summary(summary, i)
                summaries = sess.run(histogram, feed_dict=validate_feed)
                writer.add_summary(summaries, i)

                print("After %d training step(s), validation accuracy using average model is %g " % (i, acc))

            # 产生这一轮使用的一个batch的训练数据，并运行训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})
        # 在训练结束之后，在测试数据上检测神经网络模型的最终正确率
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print(("After %d training step(s), test accuracy using average model is %g" % (TRAINING_STEPS, test_acc)))
        # 关闭事件文件
        writer.close()


# 主程序
def main(argv=None):
    # 使用tensorflow方法处理数据集
    mnist = input_data.read_data_sets("../data/fashion", one_hot=True)
    train(mnist)


# TensorFlow提供的一个主程序入口
if __name__ == '__main__':
    main()

Extracting ../data/fashion/train-images-idx3-ubyte.gz
Extracting ../data/fashion/train-labels-idx1-ubyte.gz
Extracting ../data/fashion/t10k-images-idx3-ubyte.gz
Extracting ../data/fashion/t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.0696 
After 1000 training step(s), validation accuracy using average model is 0.8516 
After 2000 training step(s), validation accuracy using average model is 0.8752 
After 3000 training step(s), validation accuracy using average model is 0.8532 
After 4000 training step(s), validation accuracy using average model is 0.8894 
After 5000 training step(s), validation accuracy using average model is 0.8812 
After 6000 training step(s), validation accuracy using average model is 0.8826 
After 7000 training step(s), validation accuracy using average model is 0.8908 
After 8000 training step(s), validation accuracy using average model is 0.8864 
After 9000 training step(s), validation accuracy using average model 